In [1]:
# unsloth/DeepSeek-V3-bf16
# unsloth/DeepSeek-V3-GGUF
# unsloth/DeepSeek-V3
# unsloth/DeepSeek-R1-Zero-GGUF
# unsloth/DeepSeek-R1-Zero-BF16
# unsloth/DeepSeek-R1-Zero
# unsloth/DeepSeek-R1-GGUF
# unsloth/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Qwen-7B-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Qwen-7B-GGUF
# unsloth/DeepSeek-R1-Distill-Qwen-7B
# unsloth/DeepSeek-R1-Distill-Qwen-32B-unsloth-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Qwen-32B-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Qwen-32B-GGUF
# unsloth/DeepSeek-R1-Distill-Qwen-32B
# unsloth/DeepSeek-R1-Distill-Qwen-14B-unsloth-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Qwen-14B-unsloth-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Qwen-14B-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Qwen-14B-GGUF
# unsloth/DeepSeek-R1-Distill-Qwen-14B
# unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Qwen-1.5B-GGUF
# unsloth/DeepSeek-R1-Distill-Qwen-1.5B
# unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Llama-8B-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Llama-8B-GGUF
# unsloth/DeepSeek-R1-Distill-Llama-8B
# unsloth/DeepSeek-R1-Distill-Llama-70B-unsloth-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Llama-70B-bnb-4bit
# unsloth/DeepSeek-R1-Distill-Llama-70B-GGUF
# unsloth/DeepSeek-R1-Distill-Llama-70B
# unsloth/DeepSeek-R1-BF16
# unsloth/DeepSeek-R1

## load model and dataset

In [2]:
import torch
from unsloth import FastLanguageModel

# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
max_seq_length = 64
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    load_in_4bit = load_in_4bit,
    max_seq_length = max_seq_length,
    dtype = dtype,
)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.634 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
import os
from datasets import load_from_disk

project_name = "llm_epitope"
results_dir = os.path.join(os.path.dirname(os.getcwd()), 'models', project_name)

ds = load_from_disk(os.path.join(results_dir, 'ds'))

FileNotFoundError: Directory /home/yuan/bio/ml_epitope/models/DeepSeek-R1-Distill-Llama_epitope/ds not found

In [ ]:
# format data
catogories = ['No', 'Yes']
rank_to_label = {i:n for i,n in enumerate(catogories)}
label_to_rank = {n:i for i,n in enumerate(catogories)}

# def format_data(example, tokenizer, rank_to_label):
#     word_label = rank_to_label[example['label']]
#     example['token'] = tokenizer.encode(word_label, add_special_tokens=False)[0]
#     return example
# ds = ds.map(format_data, fn_kwargs={'tokenizer': tokenizer, 'rank_to_label': rank_to_label})

prompt = """
Here is an amino acid sequence:
{}

Is this sequence identified as Yes or No ? Answer with "Yes" or "No".

SOLUTION
The correct answer is:
{}
"""

## prepare model

In [ ]:
positivelabel = "Yes"
negativelabel = "No"

yes_token_id = tokenizer.encode("Yes", add_special_tokens=False)[0]
no_token_id = tokenizer.encode("No", add_special_tokens=False)[0]

def formatting_prompts_func(dataset_):
    # this is to fix an issue with the transformers library where the first time this function is called, it is called with a string for some reason
    if isinstance(dataset_['text'], str):
        return [" "]*100
        
    texts = []
    for i in range(len(dataset_['text'])):
        t = dataset_['text'][i]
        label = positivelabel if dataset_['label'][i] == 1 else negativelabel
        text = prompt.format(t, label)
        texts.append(text)
    return texts
# test one
formatting_prompts_func(ds['test'][:10])[0]

In [ ]:
from peft import LoftQConfig


model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "lm_head", # can easily be trained because it has only 2 tokens
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    # init_lora_weights = 'loftq',
    # loftq_config = LoftQConfig(loftq_bits = 4, loftq_iter = 1), # And LoftQ
)
print("trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
from transformers import DataCollatorForLanguageModeling
from typing import Any, Dict, List, Union

# this custom collator is needed to change the sequence labels from yes_token_id and no_token_id to 1 and 0. It also trains only on the last token of the sequence.
class DataCollatorForLastTokenLM(DataCollatorForLanguageModeling):
    def __init__(
        self,
        *args,
        mlm: bool = False,
        ignore_index: int = -100,
        **kwargs,
    ):
        super().__init__(*args, mlm=mlm, **kwargs)
        self.ignore_index = ignore_index

    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        batch = super().torch_call(examples)

        for i in range(len(examples)):
            # Find the last non-padding token
            last_token_idx = (batch["labels"][i] != self.ignore_index).nonzero()[-1].item()
            # Set all labels to ignore_index except for the last token
            batch["labels"][i, :last_token_idx] = self.ignore_index
            # The old labels for the Yes and No tokens need to be mapped to 1 and 0
            batch["labels"][i, last_token_idx] = 1 if batch["labels"][i, last_token_idx] == yes_token_id else 0


        return batch
collator = DataCollatorForLastTokenLM(tokenizer=tokenizer)

In [ ]:
import torch
from transformers import TrainingArguments

args = TrainingArguments(
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 1,
    warmup_ratio = .2,
    # num_train_epochs = 1,
    max_steps=600,
    learning_rate = 5e-7,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "cosine",
    seed = 3407,
    output_dir = os.path.join(results_dir, "outputs"),
    # report_to = "wandb",
    report_to = "none",
    group_by_length = True,
)

In [ ]:
import torch
from trl import SFTTrainer
from transformers import Trainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds['train'],
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # not needed because group_by_length is True
    args = args,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)
trainer_stats = trainer.train()

In [ ]:
from trainer_plot import TrainerPlot

TrainerPlot.loss(trainer)

In [ ]:
import os

saved_name = os.path.join(results_dir, model_name)
print(saved_name)
model.save_pretrained(saved_name)

In [ ]:
# # evaluate model using 
# import pandas as pd
# from fine_tune import FineTune

# stat = ft.trainer_predict(trainer, ds['test'])

## test

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# from unsloth import FastLanguageModel

# # max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# max_seq_length = 50
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True

# model0, tokenizer0 = FastLanguageModel.from_pretrained(
#     '/home/yuan/bio/ml_epitope/models/llama-3-8b-bnb-4bit_epitope/llama-3-8b-bnb-4bit_epitope',
#     load_in_4bit = load_in_4bit,
#     max_seq_length = max_seq_length,
#     dtype = dtype,
# )

In [ ]:
# evaluate model using 
import pandas as pd
from fine_tune import FineTune

stat = pd.DataFrame()
ft = FineTune(model, tokenizer)
chunk_size = 1000
for i in range(0, 10000, chunk_size):
    sub = ds['test'][i:i+chunk_size]
    sub=[{'text':k, 'label':v} for k,v in zip(sub['text'], sub['label'])]
    print(i, end=',')
    sub_stat = ft.predict(sub, prompt, label_to_rank)
    stat = pd.concat([stat, sub_stat])

In [ ]:
from collections import Counter
Counter(stat['predicts'])

In [ ]:
from trainer_plot import TrainerPlot
TrainerPlot.scores(stat)

In [ ]:
TrainerPlot.cm(stat)